## Iteration 7

In iteration seven, integration with Wolfram Alpha was added. Right now, the tutor model has a tool to verify the calculations, and here is an additional agent that performs verification of calculations via it's API.

The LangChain tool was not working properly due to some bug with async operations, so the custom one was added. Also, the dataset was extended with more complex questions for undergraduete topics (linear agrebra, analysis and so on).

The model was capable to solve the problems. While some tuning of system prompts was required.

In [87]:
import json
import os
import re
import jsonlines
import pandas as pd

from glob import glob
from typing import Literal
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.tools import StructuredTool
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.prebuilt import create_react_agent

In [88]:
pd.set_option('display.max_colwidth', None)

In [89]:
load_dotenv()

True

In [90]:
class MathContent(BaseModel):
    type: Literal["Задача", "Тест"]
    description: str
    solution: str
    answer: str

class JudgmentResult(BaseModel):
    has_theory_section: bool
    has_numbered_solution: bool
    has_valid_answer_format: bool
    has_valid_description_format: bool
    details: str

class VerificationResult(BaseModel):
    is_answer_correct: bool
    wolfram_result: str
    generated_answer: str
    verification_details: str

In [91]:
with jsonlines.open('dataset.jsonl') as reader:
    dataset = list(reader)

dataset[0]

{'topic': 'Лінійна алгебра (Власні числа)',
 'type': 'task',
 'description': 'Знайдіть власні числа матриці $A = \\begin{pmatrix} 4 & 1 \\\\ 2 & 3 \\end{pmatrix}$.',
 'solution': "1. Запишіть характеристичне рівняння $\\det(A - \\lambda I) = 0$. 2. Підставте значення: $\\det \\begin{pmatrix} 4-\\lambda & 1 \\\\ 2 & 3-\\lambda \\end{pmatrix} = (4-\\lambda)(3-\\lambda) - 2 = 0$. 3. Розкрийте дужки: $\\lambda^2 - 7\\lambda + 12 - 2 = 0$, що дає $\\lambda^2 - 7\\lambda + 10 = 0$. 4. Розв'яжіть квадратне рівняння за теоремою Вієта: сума коренів 7, добуток 10. Це числа 2 і 5.",
 'answer': '$\\lambda_1 = 2, \\lambda_2 = 5$'}

In [92]:
generator_model = ChatOpenAI(model="gpt-5.2", temperature=0.7)
judge_model = ChatOpenAI(model="gpt-5.2", temperature=0.1)

In [93]:
from langchain_core.tools import StructuredTool
import httpx

WOLFRAM_APP_ID = os.getenv("WOLFRAM_ALPHA_APPID")

def call_wolfram(query: str) -> str:

    base_url = "https://api.wolframalpha.com/v1/result"
    params = {
        "appid": WOLFRAM_APP_ID,
        "i": query
    }
    try:
        with httpx.Client(timeout=30) as client:
            response = client.get(base_url, params=params)
            response.raise_for_status()
            return response.text
    except Exception as e:
        return f"Error: {type(e).__name__}: {str(e)}"
    
wolfram_tool = StructuredTool.from_function(
    func=call_wolfram,
    name="wolfram_alpha",
    description="Query Wolfram Alpha for mathematical calculations. Input should be a mathematical expression or question in English."
)

In [94]:
pdf_files = glob('../../books/*.pdf')
persist_path = "../../faiss_index"
embeddings = OpenAIEmbeddings()

if os.path.exists(persist_path):
    vectorstore = FAISS.load_local(persist_path, embeddings, allow_dangerous_deserialization=True)
else:
    documents = []
    for pdf_path in pdf_files:
        loader = PyMuPDFLoader(pdf_path)
        documents.extend(loader.load())

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    vectorstore = FAISS.from_documents(chunks, embeddings)
    vectorstore.save_local(persist_path)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [95]:
TUTOR_SYSTEM_PROMPT = """Ти - вчитель математики. Проаналізуй запит користувача та надай відповідь.

Якщо користувач вказав тип контенту (Задача/Тест) - використай його. Якщо ні - обери відповідний тип самостійно.

Типи контенту:
1. Задача: 
   - description: умова задачі для обов'язкового математичного обрахунку з одним єдиним питанням
   - solution: спочатку теоретичне пояснення (формули, правила, означення), потім нумерований список кроків розв'язання
   - answer: число або вираз (без додатковий позначень)

2. Тест:
   - description: умова задачі для обов'язкового математичного обрахунку з чотирма варіантами відповіді (A, B, C, D)
   - solution: спочатку теоретичне пояснення (формули, правила, означення), потім нумерований список кроків для знаходження відповіді
   - answer: одна латинська літера (A, B, C або D)

Ніколи не задавай уточнюючих питань користувачеві.
Ніколи не створюй теоретичні задачі або тести, що не потребують обрахунків.

У тебе є доступ до інструменту Wolfram Alpha для перевірки обчислень. Завжди використовуй його для ПЕРЕВІРКИ своїх обчислень після того, як ти сам розв'яжеш задачу. Формулюй запити англійською мовою.

Відповідь надай у JSON форматі з чотирма полями: type (Задача або Тест), description, solution, answer.
Не обгортай JSON у додаткові спецсимволи."""

TUTOR_EXAMPLES = [
    {
        "input": "Тема: Площа круга, Тип: Задача",
        "output": '{"type": "Задача", "description": "Обчисліть площу круга з радіусом $r = 4$. Прийміть $\\\\pi \\\\approx 3.14$.", "solution": "Площа круга обчислюється за формулою $S = \\\\pi r^2$, де $r$ — радіус круга, а $\\\\pi$ — математична константа, що приблизно дорівнює 3.14. 1. Підставте значення радіуса у формулу: $S = 3.14 \\\\cdot 4^2$. 2. Піднесіть радіус до квадрата: $4^2 = 16$. 3. Виконайте множення: $S = 3.14 \\\\cdot 16 = 50.24$.", "answer": "50.24"}'
    },
    {
        "input": "Знайдіть корінь рівняння $2x - 5 = 7$.",
        "output": '{"type": "Задача", "description": "Знайдіть корінь рівняння $2x - 5 = 7$.", "solution": "Лінійне рівняння з однією змінною розв\\\'язується шляхом виділення невідомої. Для цього переносимо всі члени з невідомою в одну сторону, а вільні члени — в іншу, змінюючи знак на протилежний. 1. Перенесіть вільний член у праву частину: $2x = 7 + 5$. 2. Обчисліть суму: $2x = 12$. 3. Поділіть обидві частини на коефіцієнт при $x$: $x = 12 / 2 = 6$.", "answer": "6"}'
    },
    {
        "input": "Чому дорівнює $\\sqrt{144}$? \\n A) 10 \\n B) 11 \\n C) 12 \\n D) 14",
        "output": '{"type": "Тест", "description": "Чому дорівнює $\\\\sqrt{144}$? \\\\n A) 10 \\\\n B) 11 \\\\n C) 12 \\\\n D) 14", "solution": "Квадратний корінь числа $a$ — це таке невід\\\'ємне число $b$, що $b^2 = a$. Тобто $\\\\sqrt{a} = b$ означає $b \\\\cdot b = a$. 1. Потрібно знайти число, квадрат якого дорівнює 144. 2. Перевіримо варіанти: $10^2 = 100$, $11^2 = 121$, $12^2 = 144$, $14^2 = 196$. 3. Оскільки $12^2 = 144$, правильна відповідь — C.", "answer": "C"}'
    },
]


class MathTutor:
    def __init__(self, model, tools, retriever):
        self.retriever = retriever
        self.model = model
        self.tools = tools

    def invoke(self, user_input: str) -> MathContent:
        docs = self.retriever.invoke(user_input)
        context = "\n\n".join([doc.page_content for doc in docs])

        system_prompt = TUTOR_SYSTEM_PROMPT + f"\n\nКонтекст з підручника для теорітичного пояснення (якщо наявний):\n{context}"

        agent = create_react_agent(model=self.model, tools=self.tools, prompt=system_prompt)
        result = agent.invoke({"messages": [("user", user_input)]})
        output_text = result["messages"][-1].content

        json_match = re.search(r'\{.*\}', output_text, re.DOTALL)
        if json_match:
            return MathContent(**json.loads(json_match.group(0)))
        raise ValueError(f"Could not parse JSON from output: {output_text[:200]}")

In [96]:
JUDGE_SYSTEM_PROMPT = """Ти - експерт з оцінювання математичного контенту. Оціни наданий контент за критеріями.

Критерії:
1. has_theory_section: чи містить поле solution теоретичне пояснення (формули, правила, означення) перед нумерованим списком кроків
2. has_numbered_solution: чи містить поле solution нумерований список кроків
3. has_valid_answer_format:
   - для типу "Задача": відповідь має бути числом або виразом, або їх комбінацією, або відсутністью відповіді якщо таке можливо
   - для типу "Тест": відповідь має бути однією латинською літерою (A, B, C або D)
4. has_valid_description_format:
   - для типу "Задача": опис має містити умову задачі
   - для типу "Тест": опис має містити чотири варіанти відповіді (A, B, C, D)

Відповідь надай у JSON форматі з п'ятьма полями:
- has_theory_section (bool)
- has_numbered_solution (bool)
- has_valid_answer_format (bool)
- has_valid_description_format (bool)
- details (str) - коротке пояснення оцінки, особливо для критеріїв зі значенням false

Не обгортай JSON у додаткові спецсимволи."""

JUDGE_EXAMPLES = [
    {
        "input": 'Тип контенту: Задача\n\nКонтент:\n{"description": "Обчисліть площу круга з радіусом $r = 4$. Прийміть $\\\\pi \\\\approx 3.14$.", "solution": "Площа круга обчислюється за формулою $S = \\\\pi r^2$, де $r$ — радіус круга. 1. Підставте значення радіуса у формулу: $S = 3.14 \\\\cdot 4^2$. 2. Піднесіть радіус до квадрата: $4^2 = 16$. 3. Виконайте множення: $S = 3.14 \\\\cdot 16 = 50.24$.", "answer": "50.24"}',
        "output": '{"has_theory_section": true, "has_numbered_solution": true, "has_valid_answer_format": true, "has_valid_description_format": true, "details": "Усі критерії виконано: є теорія (формула S = πr²), нумеровані кроки (1-3), відповідь числом (50.24), опис з умовою задачі."}'
    },
    {
        "input": 'Тип контенту: Тест\n\nКонтент:\n{"description": "Знайдіть корінь рівняння $2x - 5 = 7$. \\n A) 1 \\n B) 6 \\n C) 12 \\n D) 2", "solution": "Лінійне рівняння розв\'язується шляхом виділення невідомої. 1. Перенесіть вільний член у праву частину: $2x = 7 + 5$. 2. Обчисліть суму: $2x = 12$. 3. Поділіть на коефіцієнт: $x = 6$. Відповідь B.", "answer": "B"}',
        "output": '{"has_theory_section": true, "has_numbered_solution": true, "has_valid_answer_format": true, "has_valid_description_format": true, "details": "Усі критерії виконано: є теорія про лінійні рівняння, 3 нумерованих кроки, відповідь однією літерою (B), опис містить варіанти A-D."}'
    },
]


class AcceptanceJudge:
    def __init__(self, model):
        self.model = model
        self.parser = PydanticOutputParser(pydantic_object=JudgmentResult)
        
        example_prompt = ChatPromptTemplate.from_messages([
            ("human", "{input}"),
            ("ai", "{output}")
        ])
        
        few_shot_prompt = FewShotChatMessagePromptTemplate(
            examples=JUDGE_EXAMPLES,
            example_prompt=example_prompt
        )
        
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", JUDGE_SYSTEM_PROMPT),
            few_shot_prompt,
            ("human", "Тип контенту: {content_type}\n\nКонтент:\n{content}")
        ])
        
        self.chain = self.prompt | self.model | self.parser

    def evaluate(self, content: MathContent) -> JudgmentResult:
        content_json = json.dumps(content.model_dump(), ensure_ascii=False)
        return self.chain.invoke({"content_type": content.type, "content": content_json})

In [97]:
VERIFIER_PROMPT = """Ти - агент перевірки математичних відповідей. Твоя задача - перевірити правильність відповіді за допомогою Wolfram Alpha.

ПРОЦЕС ПЕРЕВІРКИ:
1. Проаналізуй умову задачі
2. Переформулюй задачу у вигляді запиту до Wolfram Alpha (англійською мовою)
3. Використай інструмент wolfram_alpha для отримання точної відповіді
4. Порівняй отриману відповідь з наданою відповіддю
5. Визнач, чи співпадають відповіді (враховуй різні форми запису)

ВАЖЛИВІ НЮАНСИ:
- Для тестів (де відповідь A/B/C/D) - потрібно перевірити правильність обчислень, а не літеру
- Враховуй різні форми запису (наприклад, "6" та "6.0", "$x^2 - 9$" та "x^2 - 9")
- Якщо Wolfram Alpha не може обчислити, позначай це в verification_details

Відповідь надай у JSON форматі з полями: is_answer_correct (bool), wolfram_result (str), generated_answer (str), verification_details (str).
Не обгортай JSON у додаткові спецсимволи або markdown блоки."""

VERIFIER_INPUT_TEMPLATE = """Перевір відповідь для наступної задачі:

Тип контенту: {content_type}
Умова задачі: {description}
Розв'язання: {solution}
Відповідь для перевірки: {answer}

Використай Wolfram Alpha щоб перевірити правильність відповіді."""


class VerificationAgent:
    def __init__(self, model, wolfram_tool):
        self.model = model
        self.wolfram_tool = wolfram_tool
        self.agent = create_react_agent(
            model=self.model,
            tools=[self.wolfram_tool],
            prompt=VERIFIER_PROMPT
        )
    
    def verify(self, content: MathContent) -> VerificationResult:
        try:
            user_message = VERIFIER_INPUT_TEMPLATE.format(
                content_type=content.type,
                description=content.description,
                solution=content.solution,
                answer=content.answer
            )

            result = self.agent.invoke({"messages": [("user", user_message)]})
            output_text = result["messages"][-1].content
            
            json_match = re.search(r'\{.*\}', output_text, re.DOTALL)
            if json_match:
                verification_dict = json.loads(json_match.group(0))
                return VerificationResult(**verification_dict)
            
            return VerificationResult(
                is_answer_correct=False,
                wolfram_result="Error parsing output",
                generated_answer=content.answer,
                verification_details=f"Could not parse JSON: {output_text[:200]}"
            )
        except Exception as e:
            return VerificationResult(
                is_answer_correct=False,
                wolfram_result="Error",
                generated_answer=content.answer,
                verification_details=f"Verification error: {str(e)}"
            )

In [98]:
tutor = MathTutor(generator_model, [wolfram_tool], retriever)
judge = AcceptanceJudge(judge_model)
verifier = VerificationAgent(judge_model, wolfram_tool)

def evaluate_input(user_input: str):
    content = tutor.invoke(user_input)
    judgment = judge.evaluate(content)
    verification = verifier.verify(content)
    
    display(pd.DataFrame([
        {'field': 'user_input', 'value': user_input},
        {'field': 'tutor_type', 'value': content.type if content else ''},
        {'field': 'tutor_description', 'value': content.description if content else ''},
        {'field': 'tutor_solution', 'value': content.solution if content else ''},
        {'field': 'tutor_answer', 'value': content.answer if content else ''},
        {'field': 'judge_has_theory_section', 'value': judgment.has_theory_section if judgment else None},
        {'field': 'judge_has_numbered_solution', 'value': judgment.has_numbered_solution if judgment else None},
        {'field': 'judge_has_valid_answer_format', 'value': judgment.has_valid_answer_format if judgment else None},
        {'field': 'judge_has_valid_description_format', 'value': judgment.has_valid_description_format if judgment else None},
        {'field': 'judge_details', 'value': judgment.details if judgment else ''},
        {'field': 'verifier_is_answer_correct', 'value': verification.is_answer_correct if verification else None},
        {'field': 'verifier_wolfram_result', 'value': verification.wolfram_result if verification else ''},
        {'field': 'verifier_details', 'value': verification.verification_details if verification else ''},
    ]))

    return {
        'user_input': user_input,
        'tutor_type': content.type if content else '',
        'judge_has_theory_section': judgment.has_theory_section if judgment else None,
        'judge_has_numbered_solution': judgment.has_numbered_solution if judgment else None,
        'judge_has_valid_answer_format': judgment.has_valid_answer_format if judgment else None,
        'judge_has_valid_description_format': judgment.has_valid_description_format if judgment else None,
        'judge_details': judgment.details if judgment else '',
        'verifier_is_answer_correct': verification.is_answer_correct if verification else None,
    }

results = []

for item in dataset:
    results.append(evaluate_input(item['topic']))

for item in dataset:
    results.append(evaluate_input(item['description']))

results = pd.DataFrame(results)

C:\Users\Mykyta\AppData\Local\Temp\ipykernel_46492\970400618.py:32: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  self.agent = create_react_agent(
C:\Users\Mykyta\AppData\Local\Temp\ipykernel_46492\2069985814.py:52: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model=self.model, tools=self.tools, prompt=system_prompt)


,field,value
0,user_input,Лінійна алгебра (Власні числа)
1,tutor_type,Задача
2,tutor_description,"Знайдіть власні числа матриці A = [[2, 1], [1, 2]]."
3,tutor_solution,"Теорія: Власні числа λ квадратної матриці A знаходять із характеристичного рівняння det(A − λI) = 0, де I — одинична матриця.\n\nКроки розв’язання:\n1) Складемо матрицю A − λI:\n A − λI = [[2−λ, 1], [1, 2−λ]].\n2) Знайдемо визначник:\n det(A − λI) = (2−λ)(2−λ) − 1·1 = (2−λ)^2 − 1.\n3) Розв’яжемо рівняння det(A − λI)=0:\n (2−λ)^2 − 1 = 0\n (2−λ)^2 = 1\n 2−λ = ±1.\n4) Знайдемо λ:\n якщо 2−λ = 1, то λ = 1;\n якщо 2−λ = −1, то λ = 3.\n5) Перевірка у Wolfram Alpha (запит англійською): ""eigenvalues of {{2,1},{1,2}}"" — дає eigenvalues 3 і 1."
4,tutor_answer,"1, 3"
5,judge_has_theory_section,True
6,judge_has_numbered_solution,True
7,judge_has_valid_answer_format,True
8,judge_has_valid_description_format,True
9,judge_details,"Є теоретичне пояснення перед кроками (характеристичне рівняння det(A−λI)=0). Розв’язання подано нумерованими кроками (1–5). Формат відповіді для задачі коректний: наведено числа власних значень ""1, 3"". Опис містить умову задачі (знайти власні числа матриці)."


C:\Users\Mykyta\AppData\Local\Temp\ipykernel_46492\2069985814.py:52: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model=self.model, tools=self.tools, prompt=system_prompt)


,field,value
0,user_input,Абстрактна алгебра (Групи)
1,tutor_type,Задача
2,tutor_description,Скільки різних підгруп має циклічна група Z_36 (адитивна група цілих за модулем 36)?
3,tutor_solution,"Теорія: у циклічній групі Z_n кожній підгрупі відповідає дільник числа n, і для кожного дільника d | n існує рівно одна підгрупа порядку d. Тому кількість підгруп дорівнює кількості додатних дільників числа n.\n\nКроки розв’язання:\n1) Розкладемо 36 на прості множники: 36 = 2^2 · 3^2.\n2) Кількість додатних дільників числа 36 дорівнює (2+1)(2+1) = 9.\n3) Отже, Z_36 має 9 різних підгруп.\n\nПеревірка у Wolfram Alpha: кількість дільників 36 дорівнює 9 (DivisorSigma[0,36] = 9)."
4,tutor_answer,9
5,judge_has_theory_section,True
6,judge_has_numbered_solution,True
7,judge_has_valid_answer_format,True
8,judge_has_valid_description_format,True
9,judge_details,"Є теоретичне пояснення перед кроками (зв’язок підгруп Z_n з дільниками n), є нумеровані кроки (1–3), відповідь у форматі числа (9), опис містить умову задачі."


C:\Users\Mykyta\AppData\Local\Temp\ipykernel_46492\2069985814.py:52: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model=self.model, tools=self.tools, prompt=system_prompt)


...

,field,value
0,user_input,Чому дорівнює $5^0$? \n A) 0 \n B) 1 \n C) 5 \n D) Неможливо визначити
1,tutor_type,Тест
2,tutor_description,Чому дорівнює $5^0$?\nA) 0\nB) 1\nC) 5\nD) Неможливо визначити
3,tutor_solution,"Теорія: Для будь-якого ненульового числа $a\neq 0$ виконується правило степенів: $a^0=1$.\n\nКроки розв'язання:\n1) Маємо основу степеня $a=5$, і $5\neq 0$.\n2) Застосовуємо правило $a^0=1$.\n3) Отже, $5^0=1$, що відповідає варіанту B.\n\nПеревірка (Wolfram Alpha): запит ""5^0"" дає результат 1."
4,tutor_answer,B
5,judge_has_theory_section,True
6,judge_has_numbered_solution,True
7,judge_has_valid_answer_format,True
8,judge_has_valid_description_format,True
9,judge_details,Є теоретичне правило степенів перед кроками (a^0=1 для a≠0). Розв’язання подано нумерованими кроками (1–3). Для тесту відповідь у форматі однієї літери (B). Опис містить 4 варіанти A–D.


C:\Users\Mykyta\AppData\Local\Temp\ipykernel_46492\2069985814.py:52: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model=self.model, tools=self.tools, prompt=system_prompt)


,field,value
0,user_input,Розв'яжіть рівняння: $\frac{12}{x} + 2 = 6$.
1,tutor_type,Задача
2,tutor_description,Розв'яжіть рівняння: 12/x + 2 = 6.
3,tutor_solution,"Теорія: Рівняння з дробом виду a/x = b розв’язують, враховуючи ОДЗ: x ≠ 0, а далі виконують рівносильні перетворення (віднімають/додають однакові числа до обох частин і множать обидві частини на ненульовий вираз).\n\nКроки розв’язання:\n1) Запишемо ОДЗ: x ≠ 0.\n2) Віднімемо 2 від обох частин: 12/x = 6 − 2 = 4.\n3) Помножимо обидві частини на x (x ≠ 0): 12 = 4x.\n4) Поділимо обидві частини на 4: x = 3.\n5) Перевірка: 12/3 + 2 = 4 + 2 = 6 (вірно)."
4,tutor_answer,3
5,judge_has_theory_section,True
6,judge_has_numbered_solution,True
7,judge_has_valid_answer_format,True
8,judge_has_valid_description_format,True
9,judge_details,"Є теоретичний блок перед кроками (про ОДЗ та рівносильні перетворення), далі подано нумеровані кроки 1–5. Відповідь для задачі подана числом (3). Опис містить умову задачі (рівняння)."


C:\Users\Mykyta\AppData\Local\Temp\ipykernel_46492\2069985814.py:52: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model=self.model, tools=self.tools, prompt=system_prompt)


,field,value
0,user_input,Дано вектор $\vec{a}(3; 4)$. Знайдіть його модуль (довжину). \n A) 5 \n B) 7 \n C) 25 \n D) 12
1,tutor_type,Тест
2,tutor_description,Дано вектор \(\vec{a}(3; 4)\). Знайдіть його модуль (довжину).\nA) 5\nB) 7\nC) 25\nD) 12
3,tutor_solution,"Теорія: модуль (довжина) вектора \(\vec{a}(x;y)\) на площині обчислюється за формулою\n\[|\vec{a}|=\sqrt{x^2+y^2}.\]\n\nКроки розв’язання:\n1) Підставимо \(x=3\), \(y=4\): \(|\vec{a}|=\sqrt{3^2+4^2}\).\n2) Обчислимо: \(\sqrt{9+16}=\sqrt{25}=5\).\n3) Отже, правильний варіант — A.\n\nПеревірка (Wolfram Alpha): magnitude of vector (3,4) = 5."
4,tutor_answer,A
5,judge_has_theory_section,True
6,judge_has_numbered_solution,True
7,judge_has_valid_answer_format,True
8,judge_has_valid_description_format,True
9,judge_details,Усі критерії виконано: у solution є теоретична частина з формулою |a|=√(x²+y²) перед кроками; далі наведено нумеровані кроки 1–3; для тесту відповідь у форматі однієї літери (A); у description є 4 варіанти відповіді A–D.


In [99]:
display(results)

bool_columns = [
    'judge_has_theory_section', 
    'judge_has_numbered_solution', 
    'judge_has_valid_answer_format', 
    'judge_has_valid_description_format',
    'verifier_is_answer_correct'
]
failures = results[~results[bool_columns].all(axis=1)]

print(f"Failures: {len(failures)} / {len(results)}")
if not failures.empty:
    display(failures)

,user_input,tutor_type,judge_has_theory_section,judge_has_numbered_solution,judge_has_valid_answer_format,judge_has_valid_description_format,judge_details,verifier_is_answer_correct
0,Лінійна алгебра (Власні числа),Задача,True,True,True,True,"Є теоретичне пояснення перед кроками (характеристичне рівняння det(A−λI)=0). Розв’язання подано нумерованими кроками (1–5). Формат відповіді для задачі коректний: наведено числа власних значень ""1, 3"". Опис містить умову задачі (знайти власні числа матриці).",True
1,Абстрактна алгебра (Групи),Задача,True,True,True,True,"Є теоретичне пояснення перед кроками (зв’язок підгруп Z_n з дільниками n), є нумеровані кроки (1–3), відповідь у форматі числа (9), опис містить умову задачі.",True
2,Математичний аналіз (Границі),Задача,True,True,True,True,"Є теоретичний блок перед кроками (властивість q^n при |q|<1), далі подано нумеровані кроки 1)–4). Відповідь у форматі числа (0). Опис містить умову задачі про границю послідовності.",True
3,Лінійна алгебра (Матриці),Задача,True,True,True,True,"Є теоретичний блок із формулою визначника 3×3 перед кроками. Розв’язання подано нумерованим списком (1–6). Для типу ""Задача"" відповідь коректна (число 10). Опис містить умову задачі (знайти визначник заданої матриці).",True
4,Комплексний аналіз,Задача,True,True,True,True,"Є теоретичний блок перед кроками (формула (a+bi)(c+di) та i^2=-1), далі наведено нумеровані кроки 1–4. Формат відповіді коректний для задачі: вираз комплексного числа 26+7i. Опис містить умову задачі.",True
5,Математичний аналіз (Ряди),Задача,True,True,True,True,"Є теоретичне пояснення перед кроками (про телескопічні ряди та розклад на прості дроби), далі подано нумеровані кроки 1)–6). Відповідь для типу ""Задача"" у вигляді числа (1). Опис містить умову задачі (знайти суму нескінченного ряду).",True
6,Диференціальні рівняння,Задача,True,True,True,True,Є окремий теоретичний блок перед кроками (означення/правило для рівнянь з відокремлюваними змінними). Розв’язання подано нумерованими кроками (1–6). Відповідь для типу «Задача» є коректним виразом (y=Ce^x). Опис містить умову задачі (розв’язати dy/dx = y).,True
7,Дискретна математика (Графи),Задача,True,True,True,True,"Є теоретичний блок перед кроками (лема про рукостискання), далі наведено нумерований список (1–5). Формат відповіді для «Задача» коректний: це математичний вираз/множина значень d∈{1,3,5,7,9,11}. Опис містить повну умову задачі.",False
8,Теорія ймовірностей,Задача,True,True,True,True,"Є теоретичний блок перед кроками (класичне означення ймовірності та правило доповнення), далі наведено нумерований розв’язок (1–5). Відповідь у форматі виразу/дробу (3/4) коректна для типу «Задача». Опис містить умову задачі.",True
9,Топологія,Задача,True,True,True,True,Є теоретичний блок з означенням топології та властивостями перед кроками. Розв’язання містить нумеровані кроки (1–5). Відповідь подана числом (4). Опис містить умову задачі.,True


Failures: 1 / 60


,user_input,tutor_type,judge_has_theory_section,judge_has_numbered_solution,judge_has_valid_answer_format,judge_has_valid_description_format,judge_details,verifier_is_answer_correct
7,Дискретна математика (Графи),Задача,True,True,True,True,"Є теоретичний блок перед кроками (лема про рукостискання), далі наведено нумерований список (1–5). Формат відповіді для «Задача» коректний: це математичний вираз/множина значень d∈{1,3,5,7,9,11}. Опис містить повну умову задачі.",False
